In [17]:
import warnings
# Set the warnings to be ignored
warnings.filterwarnings('ignore')

import os
import sys
import pandas as pd 
import numpy as np 
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model  import Ridge,Lasso,RidgeCV, LassoCV, ElasticNet, ElasticNetCV, LogisticRegression
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score
import matplotlib.pyplot as plt

from pandas_profiling import ProfileReport
import seaborn as sns
import pickle

In [20]:
def output_filler(df,path):
    df["Output"] = path.split("/")[-2]
    return df

In [21]:
# How output_filler works
path = "D:/Data-Science-D-drive/Datasets-D-drive/sensor_placement/bending1/dataset1.csv"
df = pd.read_csv("D:/Data-Science-D-drive/Datasets-D-drive/sensor_placement/bending1/dataset1.csv", sep = "\t")
output_filler(df,path)

,# Task: bending1,Output
0,# Frequency (Hz): 20,bending1
1,# Clock (millisecond): 250,bending1
2,# Duration (seconds): 120,bending1
3,"# Columns: time,avg_rss12,var_rss12,avg_rss13,...",bending1
4,"0,39.25,0.43,22.75,0.43,33.75,1.30",bending1
...,...,...
479,"118750,43.33,0.47,25.00,0.00,30.00,0.00",bending1
480,"119000,43.50,0.50,25.50,0.50,30.00,0.00",bending1
481,"119250,43.50,0.50,24.75,0.43,30.00,0.00",bending1
482,"119500,43.50,0.50,24.33,0.47,30.00,0.00",bending1


In [59]:
import os

try:
    main_folder_path = "D:/Data-Science-D-drive/Datasets-D-drive/sensor_placement"

    main_folder_path = os.path.normpath(main_folder_path)
    
    try:
        contents = os.listdir(main_folder_path)
    except Exception as e:
        print(e)
    else:
        for paths in contents:
            folder_path = os.path.join(main_folder_path, paths)

            try:
                contents = os.listdir(folder_path)
            except Exception as e:
                print(e)
            else:
                path_list = []

                for item in contents:
                    path_list.append(os.path.join(folder_path, item))

                print(paths)
                for i in path_list:
                    print(i)
                print("=" * 100)

except Exception as e:
    print(e)


bending1
D:\Data-Science-D-drive\Datasets-D-drive\sensor_placement\bending1\dataset1.csv
D:\Data-Science-D-drive\Datasets-D-drive\sensor_placement\bending1\dataset2.csv
D:\Data-Science-D-drive\Datasets-D-drive\sensor_placement\bending1\dataset3.csv
D:\Data-Science-D-drive\Datasets-D-drive\sensor_placement\bending1\dataset4.csv
D:\Data-Science-D-drive\Datasets-D-drive\sensor_placement\bending1\dataset5.csv
D:\Data-Science-D-drive\Datasets-D-drive\sensor_placement\bending1\dataset6.csv
D:\Data-Science-D-drive\Datasets-D-drive\sensor_placement\bending1\dataset7.csv
bending2
D:\Data-Science-D-drive\Datasets-D-drive\sensor_placement\bending2\dataset1.csv
D:\Data-Science-D-drive\Datasets-D-drive\sensor_placement\bending2\dataset2.csv
D:\Data-Science-D-drive\Datasets-D-drive\sensor_placement\bending2\dataset3.csv
D:\Data-Science-D-drive\Datasets-D-drive\sensor_placement\bending2\dataset4.csv
D:\Data-Science-D-drive\Datasets-D-drive\sensor_placement\bending2\dataset5.csv
D:\Data-Science-D-driv

In [42]:
folder_path = "D:/Data-Science-D-drive/Datasets-D-drive/sensor_placement/bending1"

# List all files and directories in the folder
contents = os.listdir(folder_path)

path_list = []

# Store the paths
for item in contents:
    path_list.append(folder_path + "/" + item)

In [45]:
for path in path_list:
    csv_content = pd.read_csv(path, sep = ",",skiprows=4,header=0)
    csv_content = output_filler(csv_content,path)

    full_file_name = path.split("/")[-1]
    file_name = full_file_name.split(".")[-2]
    #csv_content.to_csv(folder_path + "/new_"+ file_name + ".csv")

In [44]:
csv_content

,# Columns: time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23,Output
0,0,42.00,0.00,18.50,0.50,12.00,0.00,bending1
1,250,42.00,0.00,18.00,0.00,11.33,0.94,bending1
2,500,42.75,0.43,16.75,1.79,18.25,0.43,bending1
3,750,42.50,0.50,16.75,0.83,19.00,1.22,bending1
4,1000,43.00,0.82,16.25,0.83,18.00,0.00,bending1
...,...,...,...,...,...,...,...,...
475,118750,44.25,0.83,16.25,1.30,24.00,0.00,bending1
476,119000,44.00,0.71,11.00,1.00,24.00,0.00,bending1
477,119250,44.25,0.43,10.25,1.30,22.75,0.83,bending1
478,119500,43.75,0.43,7.33,2.87,21.50,0.50,bending1
